# Bài toán 2: Hệ thống Gợi ý Nhà Tương Tự (KNN)

## Mục tiêu
Xây dựng hệ thống recommendation đơn giản sử dụng KNN để gợi ý các căn nhà có đặc điểm tương tự.

## Use Case
- Người dùng xem một căn nhà → Hệ thống gợi ý các căn tương tự
- Người dùng nhập yêu cầu (giá, diện tích, vị trí) → Hệ thống tìm nhà phù hợp

## Dataset
- File: `data/complete_house_dataset.csv`
- Số lượng: ~118,000 records

## Workflow
1. Load và clean data
2. Feature Engineering
3. Train KNN model với cosine similarity
4. 2 recommendation functions (by ID, by features)
5. Test và demo

---

## 1. Import Libraries

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors

# Cấu hình
pd.set_option('display.max_columns', None)

print("All libraries imported successfully!")

All libraries imported successfully!


## 2. Load và Prepare Data

In [3]:
# Load dataset
df = pd.read_csv('data/complete_house_dataset.csv')

print("="*80)
print("DATASET INFO")
print("="*80)
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")
print(f"\nDataset loaded successfully!")

# Preview
df.head()

DATASET INFO
Rows: 14,971
Columns: 28

Dataset loaded successfully!


,id,title,price,area,date,city,district,ward,street,lat,lng,body,rooms,toilets,floors,legal,seller_type,protection,image_count,image_thumb,width,length,pty_characteristics,owner_type,is_pro,verified,page,dateFormatted
0,165308404,"Bán Nhà Thạch Bàn 59m2 x 7 tầng,thang máy,đườn...",17500000000,59.0,1766120595000,Hà Nội,Quận Long Biên,Phường Thạch Bàn,Đường Nguyễn Văn Linh,21.027077,105.923355,"-DT:59,1m2,MT:4m đường thông,ô tô tránh nhau,k...",5.0,5.0,7.0,1.0,True,False,10,https://cdn.chotot.com/xDCe86OOiE5WCoJ1UvQSa4X...,4.0,15.0,[2],True,True,False,0,2025-12-19
1,172448163,BÁN NHÀ NGỌC THUỴ 5 TẦNG XÂY ĐẸP - NGÕ Ô TÔ ĐỖ...,7500000000,40.0,1766120565000,Hà Nội,Quận Long Biên,Phường Ngọc Thụy,Đường Ngọc Thụy,21.059816,105.863655,"- diện tích 40m2 \n- Bán nhà 5 tầng, full nội ...",3.0,4.0,5.0,1.0,True,False,8,https://cdn.chotot.com/eyVUiVEYyKCf7vUjZ-0tHD9...,NaN,NaN,"[2, 3]",True,True,False,0,2025-12-19
2,172901889,CHÍNH CHỦ CẦN BÁN GẤP NHÀ PHỐ HOA BẰNG CẦU GIẤ...,17200000000,61.0,1766120402000,Hà Nội,Quận Cầu Giấy,Phường Yên Hoà,Phố Hoa Bằng,21.025305,105.795815,🏡 Chính chủ cần bán NHÀ TRUNG TÂM YÊN HÒA \n📍...,10.0,NaN,7.0,1.0,False,False,6,https://cdn.chotot.com/5RUJ78HYuAz2rAA4VjVW4YI...,NaN,NaN,[2],NaN,False,False,0,2025-12-19
3,171753972,"7,78tỷ, 123m2, 3PN CĂN GÓC CC PACKEXIM Võ Chí ...",7850000000,123.0,1766120335000,Hà Nội,Quận Tây Hồ,Phường Phú Thượng,An Dương Vương,21.087900,105.814640,Cần Bán gấp căn hộ chung cư PACKEXIM Tây Hồ ch...,3.0,2.0,NaN,6.0,False,False,6,https://cdn.chotot.com/_dNyy_yK4IgDsdQNJ6RYSRU...,NaN,NaN,[],NaN,False,False,0,2025-12-19
4,173047068,CẦN BÁN ĐẤT CỔ LINH LONG BIÊN HÀ NỘI.,13950000000,66.0,1766120257000,Hà Nội,Quận Long Biên,Phường Long Biên,Đường Cổ Linh,21.031935,105.879370,CẦN BÁN ĐẤT CỔ LINH LONG BIÊN HÀ NỘI.\n+ Diện ...,NaN,NaN,NaN,1.0,True,False,4,https://cdn.chotot.com/XetgZAdkKl3m5tHBHMOYmUR...,4.0,16.5,[2],True,True,False,0,2025-12-19


In [4]:
# Data cleaning
print("="*80)
print("DATA CLEANING")
print("="*80)

initial_count = len(df)

# Loại bỏ duplicates
df = df.drop_duplicates()
print(f"Removed {initial_count - len(df):,} duplicates")

# Loại bỏ giá <= 0
df = df[df['price'] > 0]
print(f"Removed invalid prices")

# Loại bỏ outliers (1-99 percentile)
price_q1 = df['price'].quantile(0.01)
price_q99 = df['price'].quantile(0.99)
df = df[(df['price'] >= price_q1) & (df['price'] <= price_q99)]

if 'area' in df.columns:
    df = df[df['area'] > 0]
    area_q1 = df['area'].quantile(0.01)
    area_q99 = df['area'].quantile(0.99)
    df = df[(df['area'] >= area_q1) & (df['area'] <= area_q99)]
    print(f"Filtered area: {area_q1:.0f} - {area_q99:.0f} m²")

print(f"\nFinal dataset: {len(df):,} rows ({len(df)/initial_count*100:.1f}%)")

# Reset index
df = df.reset_index(drop=True)

DATA CLEANING
Removed 0 duplicates
Removed invalid prices
Filtered area: 16 - 561 m²

Final dataset: 14,415 rows (96.3%)


## 3. Feature Engineering cho Recommendation

In [5]:
# Tạo features mới cho recommendation
print("="*80)
print("FEATURE ENGINEERING")
print("="*80)

# Basic features
if 'area' in df.columns:
    df['price_per_sqm'] = df['price'] / df['area']
    print("Created: price_per_sqm")

if 'rooms' in df.columns and 'toilets' in df.columns:
    df['total_rooms'] = df['rooms'].fillna(0) + df['toilets'].fillna(0)
    print("Created: total_rooms")

if 'area' in df.columns and 'floors' in df.columns:
    df['area_per_floor'] = df['area'] / (df['floors'].fillna(1) + 0.1)
    print("Created: area_per_floor")

if 'lat' in df.columns and 'lng' in df.columns:
    center_lat, center_lng = 21.0285, 105.8542
    df['distance_from_center'] = np.sqrt(
        (df['lat'].fillna(center_lat) - center_lat)**2 + 
        (df['lng'].fillna(center_lng) - center_lng)**2
    )
    print("Created: distance_from_center")

print(f"\nTotal features: {df.shape[1]}")

FEATURE ENGINEERING
Created: price_per_sqm
Created: total_rooms
Created: area_per_floor
Created: distance_from_center

Total features: 32


In [6]:
# Chọn features cho recommendation
recommendation_features = [
    'area', 'price', 'rooms', 'toilets', 'floors',
    'lat', 'lng', 'width', 'length',
    'price_per_sqm', 'total_rooms', 'area_per_floor', 'distance_from_center'
]

# Filter features that exist
recommendation_features = [f for f in recommendation_features if f in df.columns]

# Handle missing values
for col in recommendation_features:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# Encode categorical features
categorical_features = ['district', 'ward', 'legal', 'seller_type']
label_encoders = {}

for col in categorical_features:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')
        le = LabelEncoder()
        df[col + '_encoded'] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le
        recommendation_features.append(col + '_encoded')
        print(f"Encoded: {col} ({len(le.classes_)} categories)")

# Tăng trọng số cho price (duplicate price features)
# Điều này làm cho price có ảnh hưởng gấp 2 lần trong similarity
df['price_weighted'] = df['price']
df['price_per_sqm_weighted'] = df['price_per_sqm']
recommendation_features.extend(['price_weighted', 'price_per_sqm_weighted'])

print(f"\nTotal recommendation features: {len(recommendation_features)}")
print(f"Features: {recommendation_features}")
print(f"Price features weighted x2 for higher priority")

Encoded: district (52 categories)
Encoded: ward (513 categories)
Encoded: legal (7 categories)
Encoded: seller_type (2 categories)

Total recommendation features: 19
Features: ['area', 'price', 'rooms', 'toilets', 'floors', 'lat', 'lng', 'width', 'length', 'price_per_sqm', 'total_rooms', 'area_per_floor', 'distance_from_center', 'district_encoded', 'ward_encoded', 'legal_encoded', 'seller_type_encoded', 'price_weighted', 'price_per_sqm_weighted']
Price features weighted x2 for higher priority


## 4. Feature Scaling & Similarity Matrix

In [7]:
# Chuẩn bị feature matrix
X = df[recommendation_features].copy()

# Replace inf values
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.median())

print("="*80)
print("FEATURE SCALING")
print("="*80)
print(f"Feature matrix shape: {X.shape}")

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"Features scaled using StandardScaler")
print(f"Scaled matrix shape: {X_scaled.shape}")

FEATURE SCALING
Feature matrix shape: (14415, 19)
Features scaled using StandardScaler
Scaled matrix shape: (14415, 19)


## 5. Build KNN-based Recommendation System

In [8]:
# Train KNN model
print("="*80)
print("TRAINING KNN MODEL")
print("="*80)

n_neighbors = 10
knn_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute', n_jobs=-1)
knn_model.fit(X_scaled)

print(f"KNN model trained with {n_neighbors} neighbors")
print(f"Metric: cosine similarity")
print(f"Algorithm: brute force")
print(f"Total samples: {X_scaled.shape[0]:,}")

TRAINING KNN MODEL
KNN model trained with 10 neighbors
Metric: cosine similarity
Algorithm: brute force
Total samples: 14,415


## 6. Recommendation Functions

In [9]:
def recommend_by_house_id(house_id, n_recommendations=5):
    """
    Gợi ý nhà tương tự dựa trên house ID (index)
    
    Parameters:
    - house_id: index của căn nhà trong dataframe
    - n_recommendations: số lượng nhà gợi ý
    
    Returns:
    - DataFrame với thông tin các nhà được gợi ý
    """
    if house_id >= len(X_scaled):
        return None
    
    # Get features của căn nhà
    house_features = X_scaled[house_id].reshape(1, -1)
    
    # Tìm neighbors
    distances, indices = knn_model.kneighbors(house_features, n_neighbors=n_recommendations+1)
    
    # Loại bỏ chính nó (index đầu tiên)
    indices = indices[0][1:]
    distances = distances[0][1:]
    
    # Chuyển distances thành similarity scores (1 - distance)
    similarity_scores = 1 - distances
    
    # Tạo result dataframe
    result = df.iloc[indices].copy()
    result['similarity_score'] = similarity_scores
    result['rank'] = range(1, len(result) + 1)
    
    return result

In [10]:
def recommend_by_features(price, area, rooms=None, toilets=None, floors=None, 
                         district=None, lat=None, lng=None, n_recommendations=5):
    """
    Gợi ý nhà dựa trên features người dùng nhập
    
    Parameters:
    - price: giá mong muốn
    - area: diện tích
    - rooms, toilets, floors: optional
    - district: optional
    - lat, lng: optional
    - n_recommendations: số lượng gợi ý
    
    Returns:
    - DataFrame với các nhà được gợi ý
    """
    # Tạo feature vector
    user_features = {}
    
    # Required features
    user_features['price'] = price
    user_features['area'] = area
    user_features['price_per_sqm'] = price / area if area > 0 else 0
    
    # Optional features
    user_features['rooms'] = rooms if rooms is not None else df['rooms'].median()
    user_features['toilets'] = toilets if toilets is not None else df['toilets'].median()
    user_features['floors'] = floors if floors is not None else df['floors'].median()
    user_features['lat'] = lat if lat is not None else 21.0285
    user_features['lng'] = lng if lng is not None else 105.8542
    user_features['width'] = df['width'].median()
    user_features['length'] = df['length'].median()
    
    # Computed features
    user_features['total_rooms'] = user_features['rooms'] + user_features['toilets']
    user_features['area_per_floor'] = area / (user_features['floors'] + 0.1)
    center_lat, center_lng = 21.0285, 105.8542
    user_features['distance_from_center'] = np.sqrt(
        (user_features['lat'] - center_lat)**2 + 
        (user_features['lng'] - center_lng)**2
    )
    
    # Encode district if provided
    if district and 'district_encoded' in recommendation_features:
        if district in label_encoders['district'].classes_:
            user_features['district_encoded'] = label_encoders['district'].transform([district])[0]
        else:
            user_features['district_encoded'] = 0
    else:
        user_features['district_encoded'] = 0
    
    # Fill missing categorical features
    for feat in recommendation_features:
        if feat.endswith('_encoded') and feat not in user_features:
            user_features[feat] = 0
    
    # Create feature vector in correct order
    user_vector = np.array([user_features.get(f, 0) for f in recommendation_features]).reshape(1, -1)
    
    # Scale
    user_vector_scaled = scaler.transform(user_vector)
    
    # Find neighbors
    distances, indices = knn_model.kneighbors(user_vector_scaled, n_neighbors=n_recommendations)
    
    indices = indices[0]
    distances = distances[0]
    similarity_scores = 1 - distances
    
    # Result
    result = df.iloc[indices].copy()
    result['similarity_score'] = similarity_scores
    
    # Bonus cho district match (tăng trọng số district)
    if district:
        district_bonus = 0.15  # Bonus 15% cho nhà cùng district
        for i, idx in enumerate(indices):
            if df.iloc[idx].get('district', '') == district:
                result.iloc[i, result.columns.get_loc('similarity_score')] += district_bonus
    
    # Re-sort theo similarity score mới
    result = result.sort_values('similarity_score', ascending=False).reset_index(drop=True)
    result['rank'] = range(1, len(result) + 1)
    
    return result

## 7. Test Recommendation System

In [ ]:
# Test 1: Recommend by House ID
print("="*80)
print("TEST 1: RECOMMEND BY HOUSE ID")
print("="*80)

test_house_id = 100
original_house = df.iloc[test_house_id]

print(f"\nORIGINAL HOUSE (ID: {test_house_id}):")
print(f"   Price: {original_house['price']:,.0f} VNĐ ({original_house['price']/1e9:.2f} tỷ)")
print(f"   Area: {original_house['area']:.0f} m²")
print(f"   Rooms: {original_house.get('rooms', 'N/A')}")
print(f"   District: {original_house.get('district', 'N/A')}")

# Get recommendations
recommendations = recommend_by_house_id(test_house_id, n_recommendations=5)

print(f"\nTOP 5 SIMILAR HOUSES:\n")
for idx, row in recommendations.iterrows():
    print(f"Rank {int(row['rank'])}: Similarity {row['similarity_score']:.4f}")
    print(f"   Price: {row['price']:,.0f} VNĐ ({row['price']/1e9:.2f} tỷ)")
    print(f"   Area: {row['area']:.0f} m²")
    print(f"   Rooms: {row.get('rooms', 'N/A')}")
    print(f"   District: {row.get('district', 'N/A')}")
    print()

TEST 1: RECOMMEND BY HOUSE ID

🏠 ORIGINAL HOUSE (ID: 100):
   Price: 24,000,000,000 VNĐ (24.00 tỷ)
   Area: 56 m²
   Rooms: 6.0
   District: Quận Nam Từ Liêm

✨ TOP 5 SIMILAR HOUSES:

Rank 1: Similarity 0.9822
   Price: 21,000,000,000 VNĐ (21.00 tỷ)
   Area: 55 m²
   Rooms: 5.0
   District: Quận Cầu Giấy

Rank 2: Similarity 0.9822
   Price: 30,000,000,000 VNĐ (30.00 tỷ)
   Area: 60 m²
   Rooms: 6.0
   District: Quận Đống Đa

Rank 3: Similarity 0.9808
   Price: 20,400,000,000 VNĐ (20.40 tỷ)
   Area: 56 m²
   Rooms: 5.0
   District: Quận Cầu Giấy

Rank 4: Similarity 0.9775
   Price: 21,000,000,000 VNĐ (21.00 tỷ)
   Area: 52 m²
   Rooms: 5.0
   District: Quận Đống Đa

Rank 5: Similarity 0.9772
   Price: 19,900,000,000 VNĐ (19.90 tỷ)
   Area: 53 m²
   Rooms: 5.0
   District: Quận Đống Đa



In [15]:
# Test 2: Recommend by User Features
print("\n" + "="*80)
print("TEST 2: RECOMMEND BY USER INPUT")
print("="*80)

# User input example
user_price = 3_000_000_000  # 5 tỷ
user_area = 70  # 80 m²
user_rooms = 3
user_toilets = 2
user_floors = 4
user_district = "Quận Hoàn Kiếm"

print(f"\nUSER REQUIREMENTS:")
print(f"   Price: {user_price:,.0f} VNĐ ({user_price/1e9:.2f} tỷ)")
print(f"   Area: {user_area} m²")
print(f"   Rooms: {user_rooms}")
print(f"   Toilets: {user_toilets}")
print(f"   Floors: {user_floors}")
print(f"   District: {user_district}")

# Get recommendations
recommendations = recommend_by_features(
    price=user_price,
    area=user_area,
    rooms=user_rooms,
    toilets=user_toilets,
    floors=user_floors,
    district=user_district,
    n_recommendations=5
)

print(f"\nTOP 5 RECOMMENDED HOUSES:\n")
for idx, row in recommendations.iterrows():
    print(f"Rank {int(row['rank'])}: Similarity {row['similarity_score']:.4f}")
    print(f"   Price: {row['price']:,.0f} VNĐ ({row['price']/1e9:.2f} tỷ)")
    print(f"   Area: {row['area']:.0f} m²")
    print(f"   Rooms: {row.get('rooms', 'N/A')}, Toilets: {row.get('toilets', 'N/A')}, Floors: {row.get('floors', 'N/A')}")
    print(f"   District: {row.get('district', 'N/A')}")
    print(f"   Title: {row.get('title', 'N/A')[:70]}...")
    print()


TEST 2: RECOMMEND BY USER INPUT

USER REQUIREMENTS:
   Price: 3,000,000,000 VNĐ (3.00 tỷ)
   Area: 70 m²
   Rooms: 3
   Toilets: 2
   Floors: 4
   District: Quận Hoàn Kiếm

TOP 5 RECOMMENDED HOUSES:

Rank 1: Similarity 0.9149
   Price: 3,000,000,000 VNĐ (3.00 tỷ)
   Area: 60 m²
   Rooms: 2.0, Toilets: 3.0, Floors: 4.0
   District: Quận Hai Bà Trưng
   Title: NHỈNH 3 TỈ - ĐẠI LA - 60m2 – VỊ TRÍ TRUNG TÂM – TIỆN ÍCH NGẬP TRÀN...

Rank 2: Similarity 0.9143
   Price: 4,300,000,000 VNĐ (4.30 tỷ)
   Area: 55 m²
   Rooms: 3.0, Toilets: 3.0, Floors: 4.0
   District: Quận Long Biên
   Title: BÁN NHÀ TỔ 4 ĐỘC LẬP CỰ KHỐI...

Rank 3: Similarity 0.9120
   Price: 3,950,000,000 VNĐ (3.95 tỷ)
   Area: 52 m²
   Rooms: 2.0, Toilets: 1.0, Floors: 4.0
   District: Quận Hà Đông
   Title: Bán nhà cấp 4-52,1m2...

Rank 4: Similarity 0.9103
   Price: 3,500,000,000 VNĐ (3.50 tỷ)
   Area: 74 m²
   Rooms: 2.0, Toilets: 2.0, Floors: 4.0
   District: Quận Hà Đông
   Title: Bán căn hộ 74m2 KDT Xa La, Kiến Hưng, H

## 8. Save Models cho API

In [12]:
import joblib
import os

# Create models directory
os.makedirs('models', exist_ok=True)

print("="*80)
print("SAVING RECOMMENDATION MODELS FOR API")
print("="*80)

# Save KNN model
knn_file = 'models/recommendation_knn.pkl'
joblib.dump(knn_model, knn_file)
print(f"KNN model saved: {knn_file}")

# Save scaler
scaler_file = 'models/recommendation_scaler.pkl'
joblib.dump(scaler, scaler_file)
print(f"Scaler saved: {scaler_file}")

# Save scaled feature matrix
X_scaled_file = 'models/recommendation_X_scaled.pkl'
joblib.dump(X_scaled, X_scaled_file)
print(f"Scaled features saved: {X_scaled_file}")

# Save dataframe (chỉ lưu columns cần thiết)
df_slim = df[['price', 'area', 'rooms', 'toilets', 'floors', 
              'lat', 'lng', 'width', 'length', 'district', 'ward', 
              'title', 'price_per_sqm', 'total_rooms', 'area_per_floor', 
              'distance_from_center']].copy()
df_file = 'models/recommendation_df.pkl'
joblib.dump(df_slim, df_file)
print(f"DataFrame saved: {df_file}")

# Save feature names
features_file = 'models/recommendation_features.pkl'
joblib.dump(recommendation_features, features_file)
print(f"Feature names saved: {features_file}")

# Save label encoders
encoders_file = 'models/recommendation_encoders.pkl'
joblib.dump(label_encoders, encoders_file)
print(f"Label encoders saved: {encoders_file}")

print(f"\nAll models saved successfully!")
print(f"Ready for API integration.")

SAVING RECOMMENDATION MODELS FOR API
KNN model saved: models/recommendation_knn.pkl
Scaler saved: models/recommendation_scaler.pkl
Scaled features saved: models/recommendation_X_scaled.pkl
DataFrame saved: models/recommendation_df.pkl
Feature names saved: models/recommendation_features.pkl
Label encoders saved: models/recommendation_encoders.pkl

All models saved successfully!
Ready for API integration.


## 8. Kết luận

**Hoàn thành Recommendation System với KNN**
- Model: K-Nearest Neighbors (cosine similarity)
- Features: 17+ features (price, area, location, rooms, engineered features)
- Functions: 
  - `recommend_by_house_id()`: Gợi ý dựa trên ID nhà
  - `recommend_by_features()`: Gợi ý dựa trên input người dùng
  
**Sẵn sàng tích hợp vào backend API!**